In [94]:
import pandas as pd
import tensorflow as tf
import transformers
from transformers import DistilBertTokenizer
import seaborn as sns
import numpy as np
import math
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import random
from sklearn.metrics import classification_report
from imblearn import over_sampling
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# from transformers import TFDistilBertFoerSequenceClassification
pd.set_option('display.max_colwidth', None)
MODEL_NAME = 'distilbert-base-uncased-finetuned-sst-2-english'
BATCH_SIZE = 16
N_EPOCHS = 3




df_items = pd.read_csv(r'C:\Users\kkaus\OneDrive - HKUST Connect\Foodpanda\Data\item_list.csv')
df_outlets = pd.read_csv(r'C:\Users\kkaus\OneDrive - HKUST Connect\Foodpanda\Data\outlet_visits.csv')
food_database = pd.read_csv(r"C:\Users\kkaus\OneDrive\Documents\GitHub\foodpanda-project\data\generic-food.csv")

df_items.shape
food_database.shape


array(['Middle Eastern', 'Malaysian', 'Singaporean', 'German', 'French',
       'Indonesian', 'Mexican', 'Spanish', 'Mediterranean', 'European',
       'British', 'Thai', 'Indian', 'Korean', 'Chinese', 'Italian',
       'American', 'Japanese', 'Cantonese', 'Hong Kong', 'Taiwanese',
       'Vietnamese', 'International'], dtype=object)



(906, 4)

In [95]:
df_nonan = df_items[df_items['primary_cuisine'].notna()]
df_clean = df_nonan[df_nonan['primary_cuisine'] .str.contains("Fast Food|Dessert|Healthy|Drinks|Pizza|Burgers|Sandwich|Halal|Noodles|Cakes & Bakery|Sichuan|Western|Chiu Chow|Shanghainese|Asian|Beijing|South American") ==  False]
df_clean.primary_cuisine.unique()
# df_sample = df_clean.sample(100000)


array(['Middle Eastern', 'Malaysian', 'Singaporean', 'German', 'French',
       'Indonesian', 'Mexican', 'Spanish', 'Mediterranean', 'European',
       'British', 'Thai', 'Indian', 'Korean', 'Chinese', 'Italian',
       'American', 'Japanese', 'Cantonese', 'Hong Kong', 'Taiwanese',
       'Vietnamese', 'International'], dtype=object)

In [96]:
df_clean["x"] = df_clean["brand_name"].fillna('') + " " + df_clean["product_title"].fillna('') + " " +  df_clean["description"].fillna('')

C:\Users\kkaus\AppData\Local\Temp/ipykernel_15344/1082737213.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["x"] = df_clean["brand_name"].fillna('') + " " + df_clean["product_title"].fillna('') + " " +  df_clean["description"].fillna('')


In [135]:
X = df_clean["x"]#.sample(100000, random_state=1)
y = df_clean["primary_cuisine"]#.sample(100000, random_state=1)

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

# X = X.to_numpy()

# Splitting

X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size = 0.80)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, test_size= 0.5)
# checking proportions

print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_valid:", X_valid.shape)
print("y_valid:", y_valid.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

X_train: (506124,)
y_train: (506124,)
X_valid: (63266,)
y_valid: (63266,)
X_test: (63266,)
y_test: (63266,)


In [136]:
type(X_train)

pandas.core.series.Series

In [99]:
# list(X_train.values)
np.unique(y)



array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22])

In [100]:
le.inverse_transform(np.unique(y))

array(['American', 'British', 'Cantonese', 'Chinese', 'European',
       'French', 'German', 'Hong Kong', 'Indian', 'Indonesian',
       'International', 'Italian', 'Japanese', 'Korean', 'Malaysian',
       'Mediterranean', 'Mexican', 'Middle Eastern', 'Singaporean',
       'Spanish', 'Taiwanese', 'Thai', 'Vietnamese'], dtype=object)

In [102]:
X_train.reshape(-1, 1)

array([['Café de Coral BBQ Pork with Rice (Small)‧Chicken Wing Served with tea/coffee/soft drink'],
       ['Praise Dining Bistro 545. Japanese Style Sauteed Diced Beef '],
       ["Daniel's Restaurant S32. BBQ Pork with Instant Noodles "],
       ...,
       ['Shing Shun Seafood Restaurant 8201. Steak Cutlet with Brown Sauce '],
       ['Supreme Sashimi Combachi Sushi (1 pc) '],
       ['Po Lam Cafe Grilled Wok-fried Beef Tenderloin Set ']],
      dtype=object)

### Class balancing

In [103]:
from collections import Counter
print(f"Training target statistics: {Counter(y_train)}")
print(f"Testing target statistics: {Counter(y_test)}")


Training target statistics: Counter({7: 230117, 12: 55672, 0: 53446, 20: 37023, 3: 37007, 21: 19498, 10: 17396, 2: 10999, 22: 9585, 8: 9381, 13: 9337, 11: 4915, 18: 2196, 14: 2139, 17: 1991, 16: 1404, 4: 1154, 5: 1027, 9: 415, 19: 412, 15: 396, 6: 314, 1: 300})
Testing target statistics: Counter({7: 28779, 12: 7055, 0: 6551, 3: 4686, 20: 4615, 21: 2402, 10: 2191, 2: 1319, 8: 1254, 22: 1216, 13: 1180, 11: 568, 18: 285, 14: 260, 17: 229, 16: 189, 4: 146, 5: 110, 19: 60, 15: 52, 1: 48, 9: 37, 6: 34})


In [104]:

# over_sampler = RandomOverSampler(random_state=1)


# X_res, y_res = over_sampler.fit_resample(X_train.reshape(-1,1), y_train)
# print(f"Training target statistics: {Counter(y_res)}")
# # print(f"Testing target statistics: {Counter(y_test)}")





In [106]:
under_sampler = RandomUnderSampler(random_state=42)
X_res1, y_res1 = under_sampler.fit_resample(X_train.reshape(-1,1), y_train)
print(f"Training target statistics: {Counter(y_res1)}")
print(f"Testing target statistics: {Counter(y_test)}")

Training target statistics: Counter({0: 300, 1: 300, 2: 300, 3: 300, 4: 300, 5: 300, 6: 300, 7: 300, 8: 300, 9: 300, 10: 300, 11: 300, 12: 300, 13: 300, 14: 300, 15: 300, 16: 300, 17: 300, 18: 300, 19: 300, 20: 300, 21: 300, 22: 300})
Testing target statistics: Counter({7: 28779, 12: 7055, 0: 6551, 3: 4686, 20: 4615, 21: 2402, 10: 2191, 2: 1319, 8: 1254, 22: 1216, 13: 1180, 11: 568, 18: 285, 14: 260, 17: 229, 16: 189, 4: 146, 5: 110, 19: 60, 15: 52, 1: 48, 9: 37, 6: 34})


In [107]:
# Getting max string length in X

MAX_LEN = X_train.apply(lambda s: len([x for x in s.split()])).max()
MAX_LEN

AttributeError: 'numpy.ndarray' object has no attribute 'apply'

## Encode with  DistilBertTokenizer

In [113]:
type(X_res1)

numpy.ndarray

In [108]:
#define a tokenizer object
tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)

In [120]:
type(X_res1.tolist())

list

In [142]:
# X_res1.shape
# list(X_valid.values)
X_train = pd.Series(X_res1)
X_valid = pd.Series(X_valid)

pandas.core.series.Series

In [143]:
#tokenize the text (padding to max sequence in batch)
train_encodings = tokenizer((X_res1.tolist()), truncation=True, padding=True)
validation_encodings = tokenizer(list(X_valid.values), truncation=True, padding=True)
test_encodings = tokenizer(list(X_test.values), truncation=True, padding=True)

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
X_train_encodings_array = np.array(train_encodings["input_ids"])
X_valid_encodings_array = np.array(validation_encodings["input_ids"])
X_test_encodings_array = np.array(test_encodings["input_ids"])

In [ ]:
X_train_encodings_array

In [ ]:
#print the first paragraph and it transformation
print(f'First paragraph: \'{X_train[:1]}\'')
print(f'Input ids: {train_encodings["input_ids"][0]}')
print(f'Attention mask: {train_encodings["attention_mask"][0]}')


## Length check


In [ ]:
pd.DataFrame(train_encodings["input_ids"]).hist();


In [ ]:
len(train_encodings["attention_mask"][0]) #max len tokenized sentence - 258



###  Turn our labels and encodings into a tf.Dataset object

For some reason this doesn't work, it sounds really cool but can't actually figure out how to utilize this

In [ ]:
# train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings),
#                                                     list(y_train)))

# test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings),
#                                                     list(y_test)))

### Putting into a feedforward neural network


In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


In [2]:
model = Sequential(name="simple_feed_forward")
model.add(tf.keras.Input(shape=(258,)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(24, activation=tf.nn.softmax))
# model.output_shape
model.summary()




Model: "simple_feed_forward"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                16576     
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 24)                1560      
                                                                 
Total params: 22,296
Trainable params: 22,296
Non-trainable params: 0
_________________________________________________________________


In [ ]:
optimizerr = tf.keras.optimizers.Adam(learning_rate=5e-5)
losss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)                     # Computes the crossentropy loss between the labels and predictions. 
model.compile(optimizer=optimizerr,                                     
              loss=losss,
              metrics=['accuracy'])

model.fit(X_train_encodings_array, y_train, 
          epochs=N_EPOCHS,
          batch_size=BATCH_SIZE)


max_seq_length 128 \
per_device_train_batch_size 16 \
learning_rate 2e-5 \
num_train_epochs 5 \

In [ ]:
# Evaluating model
model.evaluate(X_valid_encodings_array, y_valid, batch_size=BATCH_SIZE)

In [ ]:
prediction = model.predict(X_valid_encodings_array)

In [ ]:
len(prediction)

In [ ]:
guess = prediction[243]
(np.where(guess==1))


In [ ]:
y_valid[243]

In [ ]:
print(classification_report(prediction,y_valid))

### Running the model on text

In [ ]:
def predict_proba(text_list, model, tokenizer):
  """
  To get array with predicted probabilities for 0 - 23 : All primary cuisines
  :param text_list: list[str]
  :param model: transformers.models.distilbert.modeling_tf_distilbert.TFDistilBertForSequenceClassification
  :param tokenizer: transformers.models.distilbert.tokenization_distilbert.DistilBertTokenizer
  :return res: numpy.ndarray
  """
     
  encodings = tokenizer(text_list, truncation=True, padding=True)
  encodings_array = np.array(encodings["input_ids"])
  preds = model.predict(encodings_array)                                          # Gives a 0-23 0s and 1s
  # res = tf.nn.softmax(preds, axis=1).numpy()                                # No clue what it does
  encodings_array
  return preds

In [ ]:
predict_proba(list(X), model, tokenizer)